In [1]:
import cv2
import numpy as np
from PIL import Image
import pandas
import math
from generate_inputs import TrajectoryDataset
from generate_inputs import device
from generate_inputs import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

In [2]:
train_dataset = TrajectoryDataset('./data/', np.arange(1,11) , 50, 0.5, math.radians(10))
print(len(train_dataset))
train_dataset[0]['image2']

1
2
3
4
5
6
7
8
9
10
112576


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [3]:
# Verifying input dimensions

device = torch.device("cuda")
net = Net()
net = net.to(device)
print(net)

input = torch.randn(1, 2, 224, 224).to(device)
out = net(input)
print(out.shape)

x = np.stack([train_dataset[0]['image1'], train_dataset[0]['image2']])
x = np.expand_dims(x, axis=0)
print(x.shape)

Net(
  (conv_layers): Sequential(
    (0): Conv2d(2, 16, kernel_size=(5, 5), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2))
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=4096, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_feat

In [4]:
# Training X- network

import random
import torch.optim as optim
idxs = list(range(len(train_dataset)))
random.shuffle(idxs)
net_x = Net()
net_x = net_x.to(device)
optimizer = optim.Adam(net_x.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(train_dataset) * 0.8)]
test_idxs = idxs[int(len(train_dataset) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][0]).float().to(device)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_x, "net_x_4.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][0]).float().to(device)
        #print(out.shape, target.shape)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        optimizer.zero_grad()
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


Test loss: 0.30840712581157453
New lowest test loss, saved!
Train loss: 0.03131544124152012
Test loss: 0.015337267787662147
New lowest test loss, saved!
Train loss: 0.014457874197370985
Test loss: 0.012354319412329083
New lowest test loss, saved!
Train loss: 0.011967873914012581
Test loss: 0.0142848185224445
Train loss: 0.010699766565868336
Test loss: 0.012498437539159956
Train loss: 0.009823116421062096
Test loss: 0.011723797855464265
New lowest test loss, saved!
Train loss: 0.009109433977572029
Test loss: 0.008559976429894414
New lowest test loss, saved!
Train loss: 0.00854771419749071
Test loss: 0.008789769628669734
Train loss: 0.008109143336412942
Test loss: 0.008909742169291957
Train loss: 0.007688026146352284
Test loss: 0.007897516206018713
New lowest test loss, saved!
Train loss: 0.0073223200111786225


In [5]:
# Training Y- network

import random
import torch.optim as optim
idxs = list(range(len(train_dataset)))
random.shuffle(idxs)
net_y = Net()
net_y = net_y.to(device)
optimizer = optim.Adam(net_y.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(train_dataset) * 0.8)]
test_idxs = idxs[int(len(train_dataset) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_y(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][1]).float().to(device)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_y, "net_y_4.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_y(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][1]).float().to(device)
        #criterion = nn.MSELoss()
        optimizer.zero_grad()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


Test loss: 0.036622740869260914
New lowest test loss, saved!
Train loss: 0.003096956828274395
Test loss: 0.0018755317548974496
New lowest test loss, saved!
Train loss: 0.0013522884356126999
Test loss: 0.001321930050463583
New lowest test loss, saved!
Train loss: 0.0010465479877496135
Test loss: 0.0010648105598687321
New lowest test loss, saved!
Train loss: 0.0008947874318258167
Test loss: 0.0007495995425074358
New lowest test loss, saved!
Train loss: 0.0008001601794521148
Test loss: 0.0007843632469435404
Train loss: 0.0007358796158784113
Test loss: 0.0009366739944056947
Train loss: 0.0006844696376738772
Test loss: 0.0005646979827061215
New lowest test loss, saved!
Train loss: 0.0006402837195055799
Test loss: 0.000595532735239456
Train loss: 0.0006039250315170258
Test loss: 0.0006235141595117982
Train loss: 0.0005709005689067477


In [6]:
# Training Angle- network

import random
import torch.optim as optim
idxs = list(range(len(train_dataset)))
random.shuffle(idxs)
net_angle = Net()
net_angle = net_angle.to(device)
optimizer = optim.Adam(net_angle.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(train_dataset) * 0.8)]
test_idxs = idxs[int(len(train_dataset) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_angle(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][2]).float().to(device)
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_angle, "net_angle_4.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_angle(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][2]).float().to(device)
        optimizer.zero_grad()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


Test loss: 0.11844205129564041
New lowest test loss, saved!
Train loss: 0.04792369318770475
Test loss: 0.03431508694785342
New lowest test loss, saved!
Train loss: 0.025252408684162186
Test loss: 0.018228779591951096
New lowest test loss, saved!
Train loss: 0.01717309074260406
Test loss: 0.0145565411889966
New lowest test loss, saved!
Train loss: 0.013133352392007807
Test loss: 0.012710981430511525
New lowest test loss, saved!
Train loss: 0.011532220953571475
Test loss: 0.011022902507534875
New lowest test loss, saved!
Train loss: 0.010213701766255428
Test loss: 0.010938811082555547
New lowest test loss, saved!
Train loss: 0.009011764194006223
Test loss: 0.009061577583099287
New lowest test loss, saved!
Train loss: 0.007491255550761119
Test loss: 0.008336231350213741
New lowest test loss, saved!
Train loss: 0.006451935731467279
Test loss: 0.0065034132400057934
New lowest test loss, saved!
Train loss: 0.00615096394178707
